In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

config = dict(
    
    low=0.5,                    # low bandpass frequency
    high=8.0,                   # high bandpass frequency
    sim=0.6,                    # similarity threshold
    df=0.2,                     # one sided frequency delta for SNR calculation
    snr_t=2.0,                  # SNR threshold
    hr_diff=1/6,                # Maximum heart rate difference between ppg, abp
    f0_low=0.667,               # Minimum valid HR in Hz
    f0_high=3.0,                # Maximum valid HR in Hz
    abp_min_bounds=[40, 100],   # Upper and lower threshold for DBP
    abp_max_bounds=[70, 190],   # Upper and lower threshold for SBP
)

worker = BuildDatabase(
    output_dir='data-2022-10-17/',
    config=config,
    win_len=256,
    fs=125,
    samples_per_file=2500,
    samples_per_patient=500,
    max_samples=300000,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

worker.run()

In [6]:
import numpy as np

def flat_lines(x):
    for i, val in enumerate(x):
        if (i + 2) >= len(x):
            break
        if (x[i + 1] == val) & (x[i + 2] == val):
            return True
    return False

x = np.array([0, 1, 2, 2, 2, 2, 3, 4, 4, 4])
flat_lines(x)

True

In [ ]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

pd.options.display.max_rows = 2500

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

df = pd.read_csv('data-2022-10-17/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [ ]:
import os
import time
import glob
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

ppg = []
abp = []
with open('data-2022-10-17/mimic3/lines/mimic3_0000000.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        ppg.append(sample['ppg'])
        abp.append(sample['abp'])

ppg = np.array(ppg)
abp = np.array(abp)

fig = go.FigureWidget()
fig.add_scatter()
fig.update_layout(
    template='plotly_dark',
)

In [ ]:
for i in range(0, 2500):
    fig.data[0].y = ppg[i, :]
    time.sleep(0.75)

In [ ]:
# for f in $files; do mongoimport --db MIMICDB --file $f; done

# from pymongo import MongoClient

# uri = 'mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.6.0'
# client = MongoClient(uri)

# coll = client.MIMICDB.mimic3_0000067

# cursor = coll.find()
# for doc in cursor:
#     print(doc)